In [24]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [25]:
import os

os.chdir(
    r"/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/abi_hackathon_2k24"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/abi_hackathon_2k24


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)
from pandarallel import pandarallel
import re
from datetime import datetime
import gc
from sklearn.preprocessing import FunctionTransformer

# from helpers.compress import reduce_mem_usage
import pickle
import warnings
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
warnings.filterwarnings("ignore")

In [27]:
data_folder = "data"
macro_data_folder = "macro_data"
processed_data_folder = "processed_data_S2"
processed_macro = "macro_processed"
fe_data = "feature_engineered_data"

In [28]:
volume_monthly_df_best = pd.read_pickle(
    os.path.join(
        os.getcwd(), processed_data_folder, "submission_id_vol_cluster_monthly_df.pkl"
    )
)

In [29]:
volume_monthly_df_best.head()

,poc_ppg_id,ppg_name,unique_poc_id,year,month,sales_volume_hl_agg,year_month_start_date,sample_submission_flag,id_life_flag,poc_ppg_cluster
0,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,9,2.657029,2021-09-01,1.0,1.0,1_1
1,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,10,1.635095,2021-10-01,1.0,1.0,1_1
2,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,11,5.477568,2021-11-01,1.0,1.0,1_1
3,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,12,4.905285,2021-12-01,1.0,1.0,1_1
4,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2022,1,0.613161,2022-01-01,1.0,1.0,1_1


In [30]:
test_cut_date = "2023-01-01"
full_train_df = volume_monthly_df_best.loc[volume_monthly_df_best['year_month_start_date']<test_cut_date].reset_index(drop=True)
test_df =  volume_monthly_df_best.loc[volume_monthly_df_best['year_month_start_date']>=test_cut_date].reset_index(drop=True)

In [31]:
id_life = full_train_df.groupby(['poc_ppg_id'],as_index=False).agg({"year_month_start_date":'count'})

In [32]:
id_life = id_life.sort_values(['year_month_start_date'],ascending=False).reset_index(drop=True)

In [33]:
id_life.head(20)['poc_ppg_id'].unique().tolist()

['NA1__DG_CF_AF_CG_EH',
 'NA1__CJ_CZ_CG_EH',
 'BDMU__DG_CF_AF_CG_EH',
 'BJBP__DG_CF_AF_CG_EH',
 'BJAZ__CJ_DM_W_CG_T',
 'BJAZ__DG_CF_AF_CG_EH',
 'BJBA__CJ_CZ_CG_EH',
 'BJBA__DG_CF_AF_CG_EH',
 'BJBJ__DG_CF_AF_CG_EH',
 'BJBM__ET_ES_CV',
 'YHX__AR_EW_FM_DU_FN_DY',
 'BJAQ__DG_CF_AF_CG_EH',
 'BJBQ__DI_BV_FU_CE_BZ_ES_DP',
 'BJBQ__DI_BV_FU_CE_BZ_FN_DY',
 'BJBQ__ET_ES_CV',
 'BJBV__CJ_CZ_CG_EH',
 'BJBV__DG_CF_AF_CG_EH',
 'BJAZ__CJ_CZ_CG_EH',
 'BJAQ__CJ_CZ_CG_EH',
 'BJBW__DG_CF_AF_CG_EH']

In [34]:
legit_ids = id_life.loc[id_life['year_month_start_date']>=12]['poc_ppg_id'].unique().tolist()
len(legit_ids)

46119

In [35]:
temp1_df = full_train_df.loc[full_train_df['poc_ppg_id']=="NA1__DG_CF_AF_CG_EH"].reset_index(drop=True)

In [36]:
temp1_df.head()

,poc_ppg_id,ppg_name,unique_poc_id,year,month,sales_volume_hl_agg,year_month_start_date,sample_submission_flag,id_life_flag,poc_ppg_cluster
0,NA1__DG_CF_AF_CG_EH,DG CF AF CG EH,NA1,2019,1,0.373878,2019-01-01,0.0,0.0,0_1
1,NA1__DG_CF_AF_CG_EH,DG CF AF CG EH,NA1,2019,1,0.638206,2019-01-01,0.0,0.0,0_1
2,NA1__DG_CF_AF_CG_EH,DG CF AF CG EH,NA1,2019,2,0.411266,2019-02-01,0.0,0.0,0_1
3,NA1__DG_CF_AF_CG_EH,DG CF AF CG EH,NA1,2019,2,0.534693,2019-02-01,0.0,0.0,0_1
4,NA1__DG_CF_AF_CG_EH,DG CF AF CG EH,NA1,2019,3,0.336491,2019-03-01,0.0,0.0,0_1


In [37]:
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from joblib import Parallel, delayed

In [ ]:
def train_arimax_auto_arima(endog, exog, seasonal=False, m=1):
    
    # Find the best order using auto_arima
    stepwise_model = auto_arima(endog, exogenous=exog, seasonal=seasonal, m=m, trace=True, error_action='ignore', suppress_warnings=True)
    order = stepwise_model.order
    seasonal_order = stepwise_model.seasonal_order if seasonal else (0, 0, 0, 0)
    
    # Fit ARIMAX model
    model = SARIMAX(endog, exog=exog, order=order, seasonal_order=seasonal_order)
    results = model.fit(disp=False)
    return results


In [ ]:
def parallel_train_arimax_auto_arima(data, seasonal=False, m=1, n_jobs=-1):

    # Split data by entity
    grouped = data.groupby('entity')

    # Train models in parallel
    results = Parallel(n_jobs=n_jobs)(
        delayed(train_arimax_auto_arima)(
            group['endog'],
            group[['exog1', 'exog2']],
            seasonal,
            m
        ) for name, group in grouped
    )

    return results
